In [1]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

````{margin}
```{spec} Improved symbolic summation class
:id: TR-999
:status: Draft
:tags: sympy

Attempt to make AmpForm's [`PoolSum`](https://ampform.readthedocs.io/en/0.14.x/api/ampform.sympy.html#ampform.sympy.PoolSum) class suitable for handling symbolic ranges.
```
````

# Symbolic sum class

In [2]:
%pip install -q sympy==1.12

Note: you may need to restart the kernel to use updated packages.


## Problem description

In [11]:
l, m = sp.symbols("l m", integer=True, nonnegative=True)
ylm_sum = sp.Sum(
    Ynm(l, m, theta, phi),
    (m, -l, l),
    (l, 0, 5),
)
ylm_sum

Sum(Ynm(l, m, theta, phi), (m, -l, l), (l, 0, 5))

In [17]:
ynm_sum.doit()

Sum(Ynm(l, m, theta, phi), (m, -l, l), (l, 0, 5))

In [3]:
import sympy as sp
from sympy.functions.special.spherical_harmonics import Ynm

theta, phi = sp.symbols("theta phi", real=True)
Ynm(3, 2, theta, phi).expand(func=True)

-sqrt(210)*exp(2*I*phi)*cos(theta)**3/(8*sqrt(pi)) + sqrt(210)*exp(2*I*phi)*cos(theta)/(8*sqrt(pi))

In [4]:
Ynm(2, 3, theta, phi).expand(func=True)

ValueError: assoc_legendre : abs('2nd index') must be <= '1st index' (got 2, 3)